References
1. https://www.kaggle.com/code/ashishkumarak/mixtral-moe-8x7b-instruct-inference-t4-2-gpu

In [1]:
!pip install --upgrade accelerate==0.29.2 bitsandbytes==0.43.1 Faker==24.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0


In [2]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [3]:
class Conf(NamedTuple):
    debug: bool = False  
    input_dir: Path = Path("/kaggle/input")
    comp_dir: Path = input_dir / "pii-detection-removal-from-educational-data"
    temp_dir: Path = Path('/kaggle/temp')
    # write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
    working_dir: Path = Path('/kaggle/working')
    resource_dir: Path = input_dir / "libtlalpii/tlal-pii-0.1"
    data_dir: Path = resource_dir / "input"
    target_size = 30
    model_dir = input_dir / "mixtral/pytorch/8x7b-instruct-v0.1-hf/1"
    #model_dir = input_dir / "mistral/pytorch/7b-instruct-v0.1-hf/1"
    model_max_length = 4096
    seed = 9


conf = Conf()
print(conf)

Conf(debug=False, input_dir=PosixPath('/kaggle/input'), comp_dir=PosixPath('/kaggle/input/pii-detection-removal-from-educational-data'), temp_dir=PosixPath('/kaggle/temp'), working_dir=PosixPath('/kaggle/working'), resource_dir=PosixPath('/kaggle/input/libtlalpii/tlal-pii-0.1'), data_dir=PosixPath('/kaggle/input/libtlalpii/tlal-pii-0.1/input'))


In [4]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, Tesla T4
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, Tesla T4
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [5]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
sys.path.append(str(conf.input_dir / "sgcharts-ml/src"))
sys.path.append(str(conf.resource_dir / "src"))
import scml
from scml import nlp as snlp
from scml import pandasx as pdx
from mylib.datageneration import Prompter
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
scml.seed_everything(conf.seed)
tim = scml.Timer()
tim.start()

In [6]:
with open(conf.data_dir / "student_names.json") as f:
    student_names = set(json.load(f))
print(f"{len(student_names):,} student names")

104 student names


In [7]:
df = pd.read_csv(conf.data_dir / "geocode-db.csv", low_memory=False, names=["street","lat","lng"])
street_addresses = [x.title() for x in df["street"]]
print(f"{len(street_addresses):,} street addresses={street_addresses[:10]}")
street_addresses = set(street_addresses)
df.info()

11,562 street addresses=['Adis Road (Liv On Sophia)', 'Adis Road (Parc Sophia)', 'Admiralty Drive (Block 353A)', 'Admiralty Drive (Block 353B)', 'Admiralty Drive (Block 353C)', 'Admiralty Drive (Block 354A)', 'Admiralty Drive (Block 354C)', 'Admiralty Drive (Block 354D)', 'Admiralty Drive (Block 356B)', 'Admiralty Drive (Block 356C)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   street  11562 non-null  object 
 1   lat     11562 non-null  float64
 2   lng     11562 non-null  float64
dtypes: float64(2), object(1)
memory usage: 271.1+ KB


In [8]:
tokenizer = AutoTokenizer.from_pretrained(str(conf.model_dir), model_max_length=conf.model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [9]:
%%time
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
# from qlora paper:
# 4-bit NormalFloat (NF4), a new data type that is information theoretically optimal for normally distributed weights
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,  # Uses a second quantization after the first one to save an additional 0.4 bits per parameter
)
model = AutoModelForCausalLM.from_pretrained(
    str(conf.model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 1min 44s, sys: 1min 30s, total: 3min 14s
Wall time: 14min 5s


In [10]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [11]:
prompter = Prompter(target_size=conf.target_size, student_names=student_names, street_addresses=street_addresses)
res = prompter.prompt()
print(res.prompt)

[INST] Write a short essay on the following topic:
        In Education Policy and Reform, how can the integration of a learning launch tool within the design thinking process enhance the development and testing of ideas, prototypes, and products?
        Intersperse the following information in different parts of the essay:
        1. Give an example that occurred near Jurong West Street 81 (Block 824), at a school frequented by Firdaus Ahmad
        2. Give an example citing someone named FirdausAhmad
        3. Give an example citing: periscope.tv/officialfirdausiamahmad [/INST]
        


In [12]:
%%time
rows = []
for _ in tqdm(range(conf.target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.strip()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": conf.model_dir.name,
    })

  0%|          | 0/30 [00:00<?, ?it/s]2024-04-28 13:07:59.854522: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 13:07:59.854623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 13:07:59.966973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 30/30 [59:06<00:00, 118.22s/it]

CPU times: user 59min 2s, sys: 2.72 s, total: 59min 5s
Wall time: 59min 6s


In [13]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            30 non-null     object
 1   prompt          30 non-null     object
 2   street_address  30 non-null     object
 3   student_name    30 non-null     object
 4   username        30 non-null     object
 5   personal_url    30 non-null     object
 6   model           30 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB


In [14]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"In the field of architecture and urban design, visualization tools have become increasingly important for fostering innovation and collaboration in a design thinking approach. These tools allow architects, urban designers, and community members to communicate their ideas more effectively, leading to more creative and inclusive designs.\n\n One example of this can be seen near Jurong East Street 3","[INST] Write a short essay on the following topic:\n In Architecture and Urban Design, how can visualization tools foster innovation and collaboration in a design thinking approach?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Jurong East Street 32 (Block 373), at a nature park frequented by Nurul Huda\n 2. Give an example citing someone named _4807.nurul-iam--huda\n 3. Give an example citing: periscope.tv/nurulhudaxxx [/INST]\n",Jurong East Street 32 (Block 373),Nurul Huda,_4807.nurul-iam--huda,periscope.tv/nurulhudaxxx,1
1,"Sustainable urban development is a critical issue facing cities today as they work to create livable and resilient communities for their residents. One approach to fostering innovation and collaboration in this field is through the use of storytelling within the design thinking process. By using stories to understand the experiences and needs of community members, designers and urban planners can develop solutions that are more responsive and inclusive.\n\nOne example of the power of storytelling in sustainable urban development can be seen near Hougang Street 51, at a subway station frequented by Rahman bin Hassan. In this case, local residents shared stories about the challenges they faced when trying to access the subway station, including a steep staircase that made it difficult for elderly and disabled individuals to navigate. As a result, designers were able to incorporate a more accessible ramp into the station's renovation plan, making it easier for everyone in the community to use the station.\n\nAnother example of the impact of storytelling on sustainable urban development can be seen through the work of __iamRahman_xxx, an urban planner who uses social media to collect stories from community members about their experiences living in the city. By sharing these stories on his blog, located at blogspot.me/@iamrahmanhassan, he has been able to bring attention to issues such as lack of green space and transportation equity, and to advocate for solutions that address these concerns.\n\nIn addition to providing valuable insights into the needs and experiences of community members, storytelling can also help to build support for sustainable urban development projects. When people see themselves reflected in the stories being told, they are more likely to feel invested in the outcomes of the project. This can lead to increased collaboration and engagement, as well as greater overall buy-in for the project.\n\nMoreover, storytelling is also a great tool for communicating the benefits and impact of sustainable urban development project, making them more tangible and relatable to a wider audience, hence creating a larger support base and awareness.\n\nIn conclusion, storytelling is a powerful tool that can be used within the design thinking process to foster innovation and collaboration in sustainable urban development. By listening to and incorporating the stories of community members, designers and urban planners can develop solutions that are more inclusive and responsive to the needs of the community. Furthermore, storytelling can help to build support for these projects and increase engagement from the community","[INST] Write a short essay on the following topic:\n In Sustainable Urban Development, how can storytelling be used within the design thinking process to foster innovation and collaboration?\n Intersperse the following information in different parts of the ess

In [15]:
df.to_parquet(conf.working_dir / "out.parquet")
assert df.notna().all(axis=None)

In [16]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 1:13:14.623119
